In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


#Install Dependencies

In [ ]:
%pip install fastapi uvicorn pyngrok nest-asyncio

#Setup FastAPI/Ngrok

In [ ]:
from fastapi import FastAPI
from fastapi.middleware.cors import CORSMiddleware
from pyngrok import ngrok
import nest_asyncio
import uvicorn

# Allow FastAPI to run inside Colab's event loop
nest_asyncio.apply()

# Set up ngrok with your token
ngrok.set_auth_token("34AQXQUgqFjkJH9xEuP5y5xeHm2_75EaDxY3oEz1EUTgGc63S")

# Create FastAPI app
app = FastAPI()

app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"],  # for testing
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)

@app.get("/hello")
def hello():
    return {"message": "Hello from FastAPI in Colab!"}

# Create tunnel
ngrok.kill()  #makes sure multiple connections arent made, i aint paying money to have more network connections
public_url = ngrok.connect(8000)
print(f"Public URL: {public_url}")

# Run uvicorn manually inside existing loop
config = uvicorn.Config(app=app, host="0.0.0.0", port=8000)
server = uvicorn.Server(config)
await server.serve()




Public URL: NgrokTunnel: "https://dexter-hydrarch-karla.ngrok-free.dev" -> "http://localhost:8000"


INFO:     Started server process [510]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)
INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [510]


#xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx
#xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx
#Tenby10 Backend Experiment
#xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx
#xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx

In [ ]:
%pip install cerebras-cloud-sdk pandas gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.0/95.0 kB 5.3 MB/s eta 0:00:00


In [ ]:
import os
from cerebras.cloud.sdk import Cerebras
import pandas as pd
import json
import gradio as gr

api_key = os.getenv("CEREBRAS_API_KEY_JOSEPH")

# api_key = os.getenv("CEREBRAS_API_KEY_DANI")
# api_key = os.getenv("CEREBRAS_API_KEY_KEVIN")

In [ ]:
client = Cerebras(api_key=os.environ["CEREBRAS_API_KEY"])

def generate_flashcards(vocab_list, target_language="Spanish"):
    results = []

    for word in vocab_list:
        prompt = f"""
        You are a multilingual language learning assistant.
        For the term "{word}", translate it into {target_language}, and give one natural example sentence in that language.
        Respond ONLY in valid JSON with this exact structure:
        {{
            "term": "{word}",
            "translation": "...",
            "example_sentence": "..."
        }}
        """

        try:
            completion = client.chat.completions.create(
                model="qwen-3-235b-a22b-instruct-2507",  # or another available model
                messages=[
                    {"role": "system", "content": "You are a helpful multilingual assistant."},
                    {"role": "user", "content": prompt}
                ],
                max_completion_tokens=500,
                temperature=0.7,
                top_p=0.9,
                stream=False
            )

            # The returned object has the message content in a slightly different place than OpenAI
            content = completion.choices[0].message.content.strip()
            try:
                data = json.loads(content)
            except:
                # If the model returns text instead of JSON, fallback parser
                data = {
                    "term": word,
                    "translation": content.split("\n")[0] if "\n" in content else content,
                    "example_sentence": ""
                }

            results.append({
                "Grammar/Vocab": data.get("term", word),
                "Meaning": data.get("translation", ""),
                "Example Sentence": data.get("example_sentence", "")
            })

        except Exception as e:
            results.append({
                "Grammar/Vocab": word,
                "Meaning": f"Error: {e}",
                "Example Sentence": ""
            })

    return pd.DataFrame(results)

In [ ]:
def run_flashcard_generator(text, language):
    vocab_list = [w.strip() for w in text.split("\n") if w.strip()]
    df = generate_flashcards(vocab_list, language)
    csv_path = "anki_flashcards.csv"
    df.to_csv(csv_path, index=False)
    return df, csv_path

demo = gr.Interface(
    fn=run_flashcard_generator,
    inputs=[
        gr.Textbox(label="Enter vocab terms (one per line)", lines=6, placeholder="e.g.\nla organización\nel animal\nla tecnología"),
        gr.Radio(["Spanish", "French", "Japanese"], label="Target Language")
    ],
    outputs=[
        gr.Dataframe(label="Generated Flashcards"),
        gr.File(label="Download CSV")
    ],
    title="TEN by10 – AI Flashcard Generator",
    description="Generate Anki flashcards with translations and example sentences."
)

demo.launch()


It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://29d12627cbe28dfb70.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
